In [1]:
from pyspark.sql import SQLContext
from pyspark import SparkContext


data = spark.read.csv('/FileStore/tables/Crimes___2001_to_present-9be1b.csv', inferSchema=True, header=True)

In [2]:
data.columns

Out[6]: ['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

In [3]:
drop_list = ['Dates', 'DayOfWeek', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y']

data = data.select([column for column in data.columns if column not in drop_list])

In [4]:
data.show(5)

+--------+-----------+--------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
 ID|Case Number| Date| Block|IUCR| Primary Type| Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year| Updated On|Latitude|Longitude|Location|
+--------+-----------+--------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
11034701| JA366925|01/01/2001 11:00:...| 016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...| RESIDENCE| false| false| 412| 4| 8| 45| 11| null| null|2001|08/05/2017 03:50:...| null| null| null|
11227287| JB147188|10/08/2017 03:00:...| 092XX S RACINE AVE|0281|CRIM SEXUAL ASSAULT| NON-AGGRAVATED| RESIDENCE| false| false|2222| 22| 21| 73| 02| null| null|2017|02/11/2018 03:57:...| null| null| null|
11227583| JB147595|03/28/2017 02:00:...| 026XX W 79TH ST|0620| BURGLARY| UNLAWFUL ENTRY| OTHER| false| false| 835| 8| 18| 70| 05| null| null|2017|02/11/2018 03:57:...| null| null| null|
11227293| JB147230|09/09/2017 08:17:...|060XX S EBERHART AVE|0810| THEFT| OVER $500| RESIDENCE| false| false| 313| 3| 20| 42| 06| null| null|2017|02/11/2018 03:57:...| null| null| null|
11227634| JB147599|08/26/2017 10:00:...| 001XX W RANDOLPH ST|0281|CRIM SEXUAL ASSAULT| NON-AGGRAVATED| HOTEL/MOTEL| false| false| 122| 1| 42| 32| 02| null| null|2017|02/11/2018 03:57:...| null| null| null|
+--------+-----------+--------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
only showing top 5 rows

In [5]:

data.printSchema()

root
-- ID: integer (nullable = true)
-- Case Number: string (nullable = true)
-- Date: string (nullable = true)
-- Block: string (nullable = true)
-- IUCR: string (nullable = true)
-- Primary Type: string (nullable = true)
-- Description: string (nullable = true)
-- Location Description: string (nullable = true)
-- Arrest: boolean (nullable = true)
-- Domestic: boolean (nullable = true)
-- Beat: integer (nullable = true)
-- District: integer (nullable = true)
-- Ward: integer (nullable = true)
-- Community Area: integer (nullable = true)
-- FBI Code: string (nullable = true)
-- X Coordinate: integer (nullable = true)
-- Y Coordinate: integer (nullable = true)
-- Year: integer (nullable = true)
-- Updated On: string (nullable = true)
-- Latitude: double (nullable = true)
-- Longitude: double (nullable = true)
-- Location: string (nullable = true)

In [6]:
from pyspark.sql.functions import col

# by top 20 categories
data.groupBy("Primary Type") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+-------+
 Primary Type| count|
+--------------------+-------+
 THEFT|1494719|
 BATTERY|1291953|
 CRIMINAL DAMAGE| 803703|
 NARCOTICS| 728789|
 ASSAULT| 443370|
 OTHER OFFENSE| 439195|
 BURGLARY| 400016|
 MOTOR VEHICLE THEFT| 325331|
 DECEPTIVE PRACTICE| 287123|
 ROBBERY| 265757|
 CRIMINAL TRESPASS| 201692|
 WEAPONS VIOLATION| 78274|
 PROSTITUTION| 69136|
PUBLIC PEACE VIOL...| 49620|
OFFENSE INVOLVING...| 48486|
 CRIM SEXUAL ASSAULT| 29336|
 SEX OFFENSE| 26785|
INTERFERENCE WITH...| 16983|
 GAMBLING| 14571|
LIQUOR LAW VIOLATION| 14331|
+--------------------+-------+
only showing top 20 rows

In [7]:
data.groupBy("Primary Type") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+-------+
 Primary Type| count|
+--------------------+-------+
 THEFT|1494719|
 BATTERY|1291953|
 CRIMINAL DAMAGE| 803703|
 NARCOTICS| 728789|
 ASSAULT| 443370|
 OTHER OFFENSE| 439195|
 BURGLARY| 400016|
 MOTOR VEHICLE THEFT| 325331|
 DECEPTIVE PRACTICE| 287123|
 ROBBERY| 265757|
 CRIMINAL TRESPASS| 201692|
 WEAPONS VIOLATION| 78274|
 PROSTITUTION| 69136|
PUBLIC PEACE VIOL...| 49620|
OFFENSE INVOLVING...| 48486|
 CRIM SEXUAL ASSAULT| 29336|
 SEX OFFENSE| 26785|
INTERFERENCE WITH...| 16983|
 GAMBLING| 14571|
LIQUOR LAW VIOLATION| 14331|
+--------------------+-------+
only showing top 20 rows

In [8]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression

# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="Primary Type", outputCol="words", pattern="\\W")

# stop words
add_stopwords = ["http","https","amp","rt","t","c","the"] # standard stop words

stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)

# bag of words count
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)

In [9]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
label_stringIdx = StringIndexer(inputCol = "Primary Type", outputCol = "label")

In [10]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])

# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(data)
dataset = pipelineFit.transform(data)

In [11]:
dataset.show(5)

+--------+-----------+--------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+--------------------+--------------------+--------------------+-----+
 ID|Case Number| Date| Block|IUCR| Primary Type| Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year| Updated On|Latitude|Longitude|Location| words| filtered| features|label|
+--------+-----------+--------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+--------------------+--------------------+--------------------+-----+
11034701| JA366925|01/01/2001 11:00:...| 016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...| RESIDENCE| false| false| 412| 4| 8| 45| 11| null| null|2001|08/05/2017 03:50:...| null| null| null|[deceptive, pract...|[deceptive, pract...|(48,[11,12],[1.0,...| 8.0|
11227287| JB147188|10/08/2017 03:00:...| 092XX S RACINE AVE|0281|CRIM SEXUAL ASSAULT| NON-AGGRAVATED| RESIDENCE| false| false|2222| 22| 21| 73| 02| null| null|2017|02/11/2018 03:57:...| null| null| null|[crim, sexual, as...|[crim, sexual, as...|(48,[6,22,23],[1....| 15.0|
11227583| JB147595|03/28/2017 02:00:...| 026XX W 79TH ST|0620| BURGLARY| UNLAWFUL ENTRY| OTHER| false| false| 835| 8| 18| 70| 05| null| null|2017|02/11/2018 03:57:...| null| null| null| [burglary]| [burglary]| (48,[8],[1.0])| 6.0|
11227293| JB147230|09/09/2017 08:17:...|060XX S EBERHART AVE|0810| THEFT| OVER $500| RESIDENCE| false| false| 313| 3| 20| 42| 06| null| null|2017|02/11/2018 03:57:...| null| null| null| [theft]| [theft]| (48,[0],[1.0])| 0.0|
11227634| JB147599|08/26/2017 10:00:...| 001XX W RANDOLPH ST|0281|CRIM SEXUAL ASSAULT| NON-AGGRAVATED| HOTEL/MOTEL| false| false| 122| 1| 42| 32| 02| null| null|2017|02/11/2018 03:57:...| null| null| null|[crim, sexual, as...|[crim, sexual, as...|(48,[6,22,23],[1....| 15.0|
+--------+-----------+--------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+--------------------+--------------------+--------------------+-----+
only showing top 5 rows

In [12]:
### Randomly split data into training and test sets. set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 4948198
Test Dataset Count: 2119138

In [13]:
#Logistic Regression using Count Vector Features
# Build the model
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)

# Train model with Training Data
lrModel = lr.fit(trainingData)

In [14]:
predictions = lrModel.transform(testData)

predictions.filter(predictions['prediction'] == 0) \
    .select("Location Description","Primary Type","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)




+------------------------------+------------+------------------------------+-----+----------+
 Location Description|Primary Type| probability|label|prediction|
+------------------------------+------------+------------------------------+-----+----------+
 STREET| THEFT|[0.5974260535269025,0.08617...| 0.0| 0.0|
 SIDEWALK| THEFT|[0.5974260535269025,0.08617...| 0.0| 0.0|
 STREET| THEFT|[0.5974260535269025,0.08617...| 0.0| 0.0|
 GROCERY FOOD STORE| THEFT|[0.5974260535269025,0.08617...| 0.0| 0.0|
 RESIDENTIAL YARD (FRONT/BACK)| THEFT|[0.5974260535269025,0.08617...| 0.0| 0.0|
CHURCH/SYNAGOGUE/PLACE OF W...| THEFT|[0.5974260535269025,0.08617...| 0.0| 0.0|
 SMALL RETAIL STORE| THEFT|[0.5974260535269025,0.08617...| 0.0| 0.0|
 STREET| THEFT|[0.5974260535269025,0.08617...| 0.0| 0.0|
 GROCERY FOOD STORE| THEFT|[0.5974260535269025,0.08617...| 0.0| 0.0|
 DRUG STORE| THEFT|[0.5974260535269025,0.08617...| 0.0| 0.0|
+------------------------------+------------+------------------------------+-----+----------+
only showing top 10 rows

In [15]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

Out[23]: 0.983533988041183

In [16]:
# Logistic Regression using TF-IDF Features
from pyspark.ml.feature import HashingTF, IDF

# Add HashingTF and IDF to transformation
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=10000)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms

# Redo Pipeline
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, hashingTF, idf, label_stringIdx])

In [17]:
pipelineFit = pipeline.fit(data)
dataset = pipelineFit.transform(data)

### Randomly split data into training and test sets. set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)

# Build the model
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)

# Train model with Training Data
lrModel = lr.fit(trainingData)

In [18]:

predictions = lrModel.transform(testData)

predictions.filter(predictions['prediction'] == 0) \
   .select("Location Description","Primary Type","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+----------------------------+------------+------------------------------+-----+----------+
 Location Description|Primary Type| probability|label|prediction|
+----------------------------+------------+------------------------------+-----+----------+
 VEHICLE NON-COMMERCIAL| THEFT|[0.5974260535284848,0.08617...| 0.0| 0.0|
 VEHICLE NON-COMMERCIAL| THEFT|[0.5974260535284848,0.08617...| 0.0| 0.0|
 STREET| THEFT|[0.5974260535284848,0.08617...| 0.0| 0.0|
 APARTMENT| THEFT|[0.5974260535284848,0.08617...| 0.0| 0.0|
 RESIDENCE| THEFT|[0.5974260535284848,0.08617...| 0.0| 0.0|
 STREET| THEFT|[0.5974260535284848,0.08617...| 0.0| 0.0|
 GROCERY FOOD STORE| THEFT|[0.5974260535284848,0.08617...| 0.0| 0.0|
 SMALL RETAIL STORE| THEFT|[0.5974260535284848,0.08617...| 0.0| 0.0|
 SMALL RETAIL STORE| THEFT|[0.5974260535284848,0.08617...| 0.0| 0.0|
COMMERCIAL / BUSINESS OFFICE| THEFT|[0.5974260535284848,0.08617...| 0.0| 0.0|
+----------------------------+------------+------------------------------+-----+----------+
only showing top 10 rows

In [19]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

Out[28]: 0.983533988041183

In [20]:
# Naive Bayes
from pyspark.ml.classification import NaiveBayes

# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1)

# train the model
model = nb.fit(trainingData)

In [21]:
predictions = model.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
.select("Location Description","Primary Type","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+-------------------------+------------+------------------------------+-----+----------+
 Location Description|Primary Type| probability|label|prediction|
+-------------------------+------------+------------------------------+-----+----------+
 TAVERN/LIQUOR STORE| THEFT|[0.932232331479201,8.913894...| 0.0| 0.0|
 OTHER| THEFT|[0.932232331479201,8.913894...| 0.0| 0.0|
 STREET| THEFT|[0.932232331479201,8.913894...| 0.0| 0.0|
 APARTMENT| THEFT|[0.932232331479201,8.913894...| 0.0| 0.0|
 DRUG STORE| THEFT|[0.932232331479201,8.913894...| 0.0| 0.0|
 TAVERN/LIQUOR STORE| THEFT|[0.932232331479201,8.913894...| 0.0| 0.0|
 DRIVEWAY - RESIDENTIAL| THEFT|[0.932232331479201,8.913894...| 0.0| 0.0|
 DEPARTMENT STORE| THEFT|[0.932232331479201,8.913894...| 0.0| 0.0|
HOSPITAL BUILDING/GROUNDS| THEFT|[0.932232331479201,8.913894...| 0.0| 0.0|
 SCHOOL, PUBLIC, GROUNDS| THEFT|[0.932232331479201,8.913894...| 0.0| 0.0|
+-------------------------+------------+------------------------------+-----+----------+
only showing top 10 rows

In [22]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

Out[33]: 0.999993801661272